In [1]:
import pandas as pd
import numpy as np

In [3]:
# load data

#free-style wrestling
df2017_fs = pd.read_csv('df2017_fs.csv')
df2018_fs = pd.read_csv('df2018_fs.csv')

#greko-roman wrestling
df2017_gr = pd.read_csv('df2017_gr.csv')
df2018_gr = pd.read_csv('df2018_gr.csv')

#women wrestling
df2017_ww = pd.read_csv('df2017_ww.csv')
df2018_ww = pd.read_csv('df2018_ww.csv')

In [4]:
# Add new features for predict the target 

In [5]:
# Add AGE of the athlets on the WC

def agg_age(df_work):

    # add age for all competitions
    df_work['Date'] = pd.to_datetime(df_work['Date'])
    df_work['Birthday Date'] = pd.to_datetime(df_work['Birthday Date'])
    df_work['Age'] = df_work['Date'].dt.year - df_work['Birthday Date'].dt.year
    
    df_work['Age_Com'] = ((df_work['Age Group'] == 'Seniors')&
                    ((df_work['Age'] + df_work['Birthday Date'].dt.year) == df_work['Year']) &
                    (df_work['Competition'].isin(['World Championship']))).astype('int')
    # select only world championship and 
    df_work['Age_final'] = df_work['Age'] * df_work['Age_Com']

    df_work = df_work.drop(['Age', 'Age_Com'], axis=1)
    return(df_work)

In [7]:
# Add realeted AGE of the athlets 

def realeted_age(df_work):
    df_work['Birthday Date'] = pd.to_datetime(df_work['Birthday Date'])
    # add features of mounth birghtday: from 1 to 12
    df_work['Month_BirthDay'] = df_work['Birthday Date'].dt.month
    
    # add features of quarter of birghtday: from 1 to 4
    df_work['Q1'] = (df_work['Month_BirthDay'] < 4).astype('int')
    df_work['Q2'] = (((df_work['Month_BirthDay'] >= 4) & (df_work['Month_BirthDay'] <= 6)).astype('int'))* 2
    df_work['Q3'] = (((df_work['Month_BirthDay'] >= 7) & (df_work['Month_BirthDay'] <= 9)).astype('int'))*3
    df_work['Q4'] = ((df_work['Month_BirthDay'] > 9).astype('int'))* 4

    df_work['Quarter_REA'] = df_work['Q1'] + df_work['Q2'] + df_work['Q3'] + df_work['Q4']
    # drop unnecessary columns
    df_work = df_work.drop(['Q1', 'Q2', 'Q3', 'Q4'], axis=1)
    
    # add features of birght day: the first (0) and the second (1) half of year
    df_work['Half_REA'] = (df_work['Month_BirthDay'] >= 7).astype('int')
    
    return(df_work)

In [8]:
# Add Cadets Results

def cadets(df_work):
# win competition
    df_work['Cadet_Win'] = ((df_work['Age Group'] == 'Cadets') & 
                     (df_work['Competition'].isin(['World Championship', 'European Championship', 'Asian Championship',
                                                    'European Games', 'Asian Games', 'Pan American Championship',
                                        'Pan American Games', 'African Championship', 'Oceania Championship'])) & 
                     (df_work['Rank'] == 1)).astype('int')
# success competition, 1-3 rank
    df_work['Cadet_Success'] = ((df_work['Age Group'] == 'Cadets') & 
                     (df_work['Competition'].isin(['World Championship', 'European Championship', 'Asian Championship',
                                                    'European Games', 'Asian Games', 'Pan American Championship',
                                        'Pan American Games', 'African Championship', 'Oceania Championship'])) & 
                    (df_work['Rank'] <= 3)).astype('int')
#take a part 
    df_work['Cadet_Part'] = ((df_work['Age Group'] == 'Cadets') & 
                     (df_work['Competition'].isin(['World Championship', 'European Championship', 'Asian Championship',
                                                  'European Games', 'Asian Games', 'Pan American Championship',
                                       'Pan American Games', 'African Championship', 'Oceania Championship'])) & 
                    (df_work['Rank'] >= 1)).astype('int')
    return(df_work)

In [9]:
# Add Juniors Results

def junior(df_work):
# win competition
    df_work['Junior_Win'] = ((df_work['Age Group'] == 'Juniors') & 
                     (df_work['Competition'].isin(['World Championship', 'European Championship', 'Asian Championship',
                                                    'European Games', 'Asian Games', 'Pan American Championship',
                                        'Pan American Games', 'African Championship', 'Oceania Championship'])) & 
                     (df_work['Rank'] == 1)).astype('int')
# success competition, 1-3 rank
    df_work['Junior_Success'] = ((df_work['Age Group'] == 'Juniors') & 
                     (df_work['Competition'].isin(['World Championship', 'European Championship', 'Asian Championship',
                                                    'European Games', 'Asian Games', 'Pan American Championship',
                                        'Pan American Games', 'African Championship', 'Oceania Championship'])) & 
                    (df_work['Rank'] <= 3)).astype('int')
#take a part 
    df_work['Junior_Part'] = ((df_work['Age Group'] == 'Juniors') & 
                     (df_work['Competition'].isin(['World Championship', 'European Championship', 'Asian Championship',
                                                  'European Games', 'Asian Games', 'Pan American Championship',
                                       'Pan American Games', 'African Championship', 'Oceania Championship'])) & 
                    (df_work['Rank'] >= 1)).astype('int')
    return(df_work)

In [10]:
# Add Seniors Results Before World Championship at the continental Championship (Europe, Asia etc.) 

def senior_cc(df_work):
# win competition
    df_work['Date'] = pd.to_datetime(df_work['Date'])
    df_work['Senior_Win_CC'] =  ((df_work['Age Group'] == 'Seniors') & 
                     (df_work['Competition'].isin(['European Championship', 'Asian Championship',
                                                  'European Games', 'Asian Games', 'Pan American Championship',
                                       'Pan American Games', 'African Championship', 'Oceania Championship'])) & 
                     (df_work['Date'].dt.year < df_work['Year']) &
                     (df_work['Rank'] == 1)).astype('int')
# success competition, 1-3 rank
    df_work['Senior_Success_CC'] = ((df_work['Age Group'] == 'Seniors') & 
                     (df_work['Competition'].isin(['European Championship', 'Asian Championship',
                                                  'European Games', 'Asian Games', 'Pan American Championship',
                                       'Pan American Games', 'African Championship', 'Oceania Championship'])) & 
                     (df_work['Date'].dt.year < df_work['Year']) &
                     (df_work['Rank'] <= 3)).astype('int')
#take a part
    df_work['Senior_Part_CC'] =  ((df_work['Age Group'] == 'Seniors') & 
                     (df_work['Competition'].isin(['European Championship', 'Asian Championship',
                                                  'European Games', 'Asian Games', 'Pan American Championship',
                                       'Pan American Games', 'African Championship', 'Oceania Championship'])) & 
                     (df_work['Date'].dt.year < df_work['Year']) &
                     (df_work['Rank'] >= 1)).astype('int')
    return(df_work)

In [11]:
# Add Seniors Results Before World Championship at the World Championship

def senior_wc(df_work):
# win competition
    df_work['Date'] = pd.to_datetime(df_work['Date'])
    df_work['Senior_Win_WC'] =  ((df_work['Age Group'] == 'Seniors') & 
                     (df_work['Competition'] == 'World Championship') & 
                     (df_work['Date'].dt.year < df_work['Year']) &
                     (df_work['Rank'] == 1)).astype('int')
# success competition, 1-3 rank
    df_work['Senior_Success_WC'] =  ((df_work['Age Group'] == 'Seniors') & 
                     (df_work['Competition'] == 'World Championship') & 
                     (df_work['Date'].dt.year < df_work['Year']) &
                     (df_work['Rank'] <= 3)).astype('int')
#take a part
    df_work['Senior_Part_WC'] =  ((df_work['Age Group'] == 'Seniors') & 
                     (df_work['Competition'] == 'World Championship') & 
                     (df_work['Date'].dt.year < df_work['Year']) &
                     (df_work['Rank'] >= 1)).astype('int')
    return(df_work)

In [12]:
# add target for prediction

def target(df_work):
    
    # the first target - rank
    df_work['Date'] = pd.to_datetime(df_work['Date'])
    # add 0/1 for world championsips for forecasting
    df_work['Rank_Com'] = ((df_work['Age Group'] == 'Seniors')&
                    (df_work['Date'].dt.year == df_work['Year']) &
                    (df_work['Competition'] =='World Championship')).astype('int')
    # add cureent rank for reults 
    df_work['target_rank'] = df_work['Rank_Com'] * df_work['Rank']

    #the second target rank group: medalist(from 1 to 3), middle (from 5 to 10), outsiders (from 11 to end)
    df_work['target_medalist'] = ((df_work['target_rank'] >= 1) & (df_work['target_rank'] < 4)).astype('int')
    df_work['target_middle'] = (((df_work['target_rank'] >= 4) & (df_work['target_rank'] <= 10)).astype('int'))*2
    df_work['target_outsider'] = (((df_work['target_rank'] > 10)).astype('int'))*3
    
    # medalist - 1, middle - 2, outsider - 3
    df_work['target_group'] = df_work['target_medalist'] + df_work['target_middle'] + df_work['target_outsider']
    
    # drop unnecessary columns
    df_work = df_work.drop(['target_medalist', 'target_middle', 'target_outsider'], axis=1)
    
    # drop unnecessary columns
    df_work = df_work.drop(['Rank_Com'], axis=1)
    return(df_work)

In [13]:
def all_features(df_work):
    df_work = agg_age(df_work)
    df_work = realeted_age(df_work)
    df_work = cadets(df_work)
    df_work = junior(df_work)
    df_work = senior_cc(df_work)
    df_work = senior_wc(df_work)
    df_work = target(df_work)
    return(df_work)

In [14]:
# create final df for forecasting

# use groupby for the column 'Name'

def final_df(df):
    df_final = df.groupby(['Name']).agg({
        'Age_final': ['sum'],
        'Month_BirthDay': ['median'],
        'Quarter_REA': ['median'],
        'Half_REA': ['median'],
        'Cadet_Win': ['sum'],
        'Cadet_Success': ['sum'],
        'Cadet_Part': ['sum'],
        'Junior_Win': ['sum'],
        'Junior_Success': ['sum'],
        'Junior_Part': ['sum'],
        'Senior_Win_CC': ['sum'],
        'Senior_Success_CC': ['sum'],
        'Senior_Part_CC': ['sum'],
        'Senior_Win_WC' : ['sum'],
        'Senior_Success_WC' : ['sum'],
        'Senior_Part_WC': ['sum'],
        'target_rank' : ['sum'],
        'target_group': ['sum'],
    }).reset_index()

# chouse only necessary columns' title 
    df_final.columns = df_final.columns.get_level_values(0)
    return(df_final)

In [15]:
# the first stage of preparing df
df_2017_fs_start = all_features(df2017_fs)
df_2018_fs_start = all_features(df2018_fs)

df_2017_gr_start = all_features(df2017_gr)
df_2018_gr_start = all_features(df2018_gr)

df_2017_ww_start = all_features(df2017_ww)
df_2018_ww_start = all_features(df2018_ww)

In [16]:
# the second stage of preparing of df
df_2017_fs_finish = final_df(df_2017_fs_start)
df_2018_fs_finish = final_df(df_2018_fs_start)

df_2017_gr_finish = final_df(df_2017_gr_start)
df_2018_gr_finish = final_df(df_2018_gr_start)

df_2017_ww_finish = final_df(df_2017_ww_start)
df_2018_ww_finish = final_df(df_2018_ww_start)

In [17]:
df_2018_ww_finish.head()

,Name,Age_final,Month_BirthDay,Quarter_REA,Half_REA,Cadet_Win,Cadet_Success,Cadet_Part,Junior_Win,Junior_Success,Junior_Part,Senior_Win_CC,Senior_Success_CC,Senior_Part_CC,Senior_Win_WC,Senior_Success_WC,Senior_Part_WC,target_rank,target_group
0,"Acosta Acosta, Maria Jose",27,11,4,1,0,0,0,0,0,1,0,4,5,0,0,2,18.0,3
1,"Adar, Yasemin",27,12,4,1,0,0,0,0,0,2,2,2,6,1,1,5,2.0,1
2,"Adekuoroye, Odunayo Folasade",25,10,4,1,0,0,0,0,0,0,2,3,3,0,2,2,9.0,2
3,"Adeniyi Aminat, Oluwafunmilayo",25,4,2,0,0,0,0,1,1,1,2,3,3,0,0,1,17.0,3
4,"Alieva, Sabira",23,9,3,1,0,0,0,3,5,6,0,0,1,0,0,0,22.0,3


In [18]:
# save df to csv files
df_2017_fs_finish.to_csv('df_2017_fs_finish.csv')
df_2018_fs_finish.to_csv('df_2018_fs_finish.csv')

df_2017_gr_finish.to_csv('df_2017_gr_finish.csv')
df_2018_gr_finish.to_csv('df_2018_gr_finish.csv')

df_2017_ww_finish.to_csv('df_2017_ww_finish.csv')
df_2018_ww_finish.to_csv('df_2018_ww_finish.csv')